In [2]:
from datasets import load_dataset

In [3]:
"""
Load dataset.
"""
dataset = load_dataset("yelp_review_full")
print(dataset)

Generating test split: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 282191.98 examples/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})
